In [23]:
import pandas as pd 
from datetime import datetime, date, time, timedelta

 
df = pd.read_csv("crime.csv") 
df.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [24]:

df['DATE'], df['TIME'] = df['FIRST_OCCURRENCE_DATE'].str.split(' ', 1).str


df['DATE'] = pd.to_datetime(df['DATE'])
df['day_of_week'] = df['DATE'].dt.day_name()
df

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,DATE,TIME,day_of_week
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,...,-104.809881,39.773188,5,521,montbello,1,0,2016-06-15,11:31:00 PM,Wednesday
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,...,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0,2017-10-11,12:30:00 PM,Wednesday
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,...,-104.957381,39.663490,3,314,wellshire,1,0,2016-03-04,8:00:00 PM,Friday
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,...,-104.941440,39.702698,3,312,belcaro,1,0,2018-01-30,7:20:00 PM,Tuesday
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,...,-104.955370,39.717107,3,311,cherry-creek,1,0,2017-06-22,8:53:00 PM,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526095,2019688353,2019688353121200,1212,0,robbery-car-jacking,robbery,10/27/2019 11:45:00 PM,NaN,10/28/2019 12:08:00 AM,E COLFAX AVE / N XENIA ST,...,-104.885821,39.740155,2,223,east-colfax,1,0,2019-10-27,11:45:00 PM,Sunday
526096,2019688368,2019688368540100,5401,0,traffic-accident-hit-and-run,traffic-accident,10/28/2019 12:20:00 AM,NaN,10/28/2019 12:35:00 AM,1959 S BROADWAY ST,...,-104.987794,39.681001,3,313,overland,0,1,2019-10-28,12:20:00 AM,Monday
526097,2019688404,2019688404131302,1313,2,assault-dv,other-crimes-against-persons,10/28/2019 1:13:00 AM,NaN,10/28/2019 3:26:00 AM,7030 N TOWER RD,...,-104.770112,39.824869,5,522,dia,1,0,2019-10-28,1:13:00 AM,Monday
526098,2019688441,2019688441131302,1313,2,assault-dv,other-crimes-against-persons,10/28/2019 2:10:00 AM,NaN,10/28/2019 4:25:00 AM,3831 N SHOSHONE ST,...,-105.009777,39.769921,1,111,sunnyside,1,0,2019-10-28,2:10:00 AM,Monday


In [25]:
df_property = df[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID']]
df_property = df_property.loc[(df_property.OFFENSE_CATEGORY_ID == 'arson')|(df_property.OFFENSE_CATEGORY_ID == 'theft-from-motor-vehicle')|(df_property.OFFENSE_CATEGORY_ID == 'larceny')|(df_property.OFFENSE_CATEGORY_ID == 'burglary')|(df_property.OFFENSE_CATEGORY_ID == 'auto-theft')]
df_property_am =  df_property.loc[df_property.TIME.str[-2:] == "AM" ]
df_property_am['TIME'] = [x[:-3] for x in df_property_am['TIME']]
df_property_am['TIME'] =  pd.to_datetime(df_property_am['TIME'], format='%H:%M:%S')
# df_property_am['TIME'] = [d.time() for d in df_property_am['TIME']]
df_property_am

/Users/missybueno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/missybueno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,OFFENSE_CATEGORY_ID,TIME,day_of_week,NEIGHBORHOOD_ID
9,theft-from-motor-vehicle,1900-01-01 12:55:00,Wednesday,highland
11,theft-from-motor-vehicle,1900-01-01 05:25:00,Sunday,ruby-hill
15,theft-from-motor-vehicle,1900-01-01 02:00:00,Sunday,highland
28,auto-theft,1900-01-01 12:00:00,Saturday,west-colfax
33,auto-theft,1900-01-01 07:37:00,Monday,dia
...,...,...,...,...
526067,larceny,1900-01-01 03:00:00,Friday,kennedy
526070,theft-from-motor-vehicle,1900-01-01 10:00:00,Wednesday,capitol-hill
526076,larceny,1900-01-01 09:00:00,Sunday,cbd
526092,auto-theft,1900-01-01 10:00:00,Sunday,bear-valley


In [26]:
df_property = df_property[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID']]
df_property = df_property.loc[(df_property.OFFENSE_CATEGORY_ID == 'arson')|(df_property.OFFENSE_CATEGORY_ID == 'theft-from-motor-vehicle')|(df_property.OFFENSE_CATEGORY_ID == 'larceny')|(df_property.OFFENSE_CATEGORY_ID == 'burglary')|(df_property.OFFENSE_CATEGORY_ID == 'auto-theft')]
df_property_pm =  df_property.loc[df_property.TIME.str[-2:] == "PM" ]
df_property_pm['TIME'] = [x[:-3] for x in df_property_pm['TIME']]
df_property_pm['TIME'] =  pd.to_datetime(df_property_pm['TIME'], format='%H:%M:%S')
df_property_pm['TIME'] = df_property_pm['TIME'] + timedelta(hours=12)
# df_property_pm['TIME'] = [d.time() for d in df_property_pm['TIME']]
df_property_pm

/Users/missybueno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/missybueno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/missybueno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,OFFENSE_CATEGORY_ID,TIME,day_of_week,NEIGHBORHOOD_ID
1,larceny,1900-01-02 00:30:00,Wednesday,gateway-green-valley-ranch
2,theft-from-motor-vehicle,1900-01-01 20:00:00,Friday,wellshire
3,larceny,1900-01-01 19:20:00,Tuesday,belcaro
4,larceny,1900-01-01 20:53:00,Thursday,cherry-creek
6,theft-from-motor-vehicle,1900-01-02 00:15:00,Thursday,villa-park
...,...,...,...,...
526046,theft-from-motor-vehicle,1900-01-01 18:30:00,Tuesday,cole
526081,larceny,1900-01-01 18:00:00,Sunday,north-capitol-hill
526084,auto-theft,1900-01-01 16:30:00,Sunday,sunnyside
526088,theft-from-motor-vehicle,1900-01-01 19:00:00,Friday,speer


In [27]:
final_property = pd.concat([df_property_am, df_property_pm], ignore_index=True, sort =False)

block1 = '09/19/18 04:00:00'
block2 = '09/19/18 08:00:00'
block3 = '09/19/18 12:00:00'
block4 = '09/19/18 16:00:00'
block5 = '09/19/18 20:00:00'
block6 = '09/19/18 23:59:00'
datetime_object1 = datetime.strptime(block1, '%m/%d/%y %H:%M:%S')
datetime_object2 = datetime.strptime(block2, '%m/%d/%y %H:%M:%S')
datetime_object3 = datetime.strptime(block3, '%m/%d/%y %H:%M:%S')
datetime_object4 = datetime.strptime(block4, '%m/%d/%y %H:%M:%S')
datetime_object5 = datetime.strptime(block5, '%m/%d/%y %H:%M:%S')
datetime_object6 = datetime.strptime(block6, '%m/%d/%y %H:%M:%S')


In [28]:
final_property.loc[final_property['TIME'].dt.time <= datetime_object1.time(), 'TIME'] = datetime_object1

final_property.loc[(final_property['TIME'].dt.time > datetime_object1.time())&(final_property['TIME'].dt.time <= datetime_object2.time()), 'TIME'] = datetime_object2

final_property.loc[(final_property['TIME'].dt.time > datetime_object2.time())&(final_property['TIME'].dt.time <= datetime_object3.time()), 'TIME'] = datetime_object3

final_property.loc[(final_property['TIME'].dt.time > datetime_object3.time())&(final_property['TIME'].dt.time <= datetime_object4.time()), 'TIME'] = datetime_object4

final_property.loc[(final_property['TIME'].dt.time > datetime_object4.time())&(final_property['TIME'].dt.time <= datetime_object5.time()), 'TIME'] = datetime_object5

final_property.loc[(final_property['TIME'].dt.time > datetime_object5.time())&(final_property['TIME'].dt.time <= datetime_object6.time()), 'TIME'] = datetime_object6



final_property['TIME'] = [d.time() for d in final_property['TIME']]
final_property

,OFFENSE_CATEGORY_ID,TIME,day_of_week,NEIGHBORHOOD_ID
0,theft-from-motor-vehicle,16:00:00,Wednesday,highland
1,theft-from-motor-vehicle,08:00:00,Sunday,ruby-hill
2,theft-from-motor-vehicle,04:00:00,Sunday,highland
3,auto-theft,12:00:00,Saturday,west-colfax
4,auto-theft,08:00:00,Monday,dia
...,...,...,...,...
148113,theft-from-motor-vehicle,20:00:00,Tuesday,cole
148114,larceny,20:00:00,Sunday,north-capitol-hill
148115,auto-theft,20:00:00,Sunday,sunnyside
148116,theft-from-motor-vehicle,20:00:00,Friday,speer


In [33]:
property_csv = final_property[['day_of_week','NEIGHBORHOOD_ID', 'TIME']]
property_csv = property_csv.sample(5000)
property_csv.to_csv('Property.csv')


!python apriori.py -f 'Property.csv' -s 0.0005

item: ('belcaro', 'Monday') , 0.001
item: ('chaffee-park', '08:00:00') , 0.001
item: ('Sunday', 'southmoor-park') , 0.001
item: ('Thursday', 'civic-center') , 0.001
item: ('12:00:00', 'whittier') , 0.001
item: ('bear-valley', '08:00:00') , 0.001
item: ('Wednesday', 'whittier') , 0.001
item: ('southmoor-park', '16:00:00') , 0.001
item: ('Tuesday', 'washington-park') , 0.001
item: ('Sunday', 'platt-park') , 0.001
item: ('windsor', '04:00:00') , 0.001
item: ('belcaro', '23:59:00') , 0.001
item: ('Friday', 'overland') , 0.001
item: ('Friday', 'sun-valley') , 0.001
item: ('clayton', 'Thursday') , 0.001
item: ('12:00:00', 'washington-park') , 0.001
item: ('Thursday', 'wellshire') , 0.001
item: ('Sunday', 'belcaro') , 0.001
item: ('Thursday', 'congress-park') , 0.001
item: ('washington-park', 'Monday') , 0.001
item: ('20:00:00', 'city-park') , 0.001
item: ('Saturday', 'whittier') , 0.001
item: ('chaffee-park', '23:59:00') , 0.001
item: ('Sunday', 'wellshire') , 0.001
item: ('Saturday', 'regis